<a href="https://colab.research.google.com/github/anareisier/projeto-instagram/blob/main/ALGORTIMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pdb; 
pdb.set_trace()

In [ ]:

#Bibliotecas
import matplotlib.pyplot as plt
import numpy as np

#Dados

mesH=[744,672,744,720,744,720,744,744,720,744,720,744]
mes=["Janeiro","Fevereiro","Março","Abril","Maio","Junho","Julho","Agosto","Setembro","Outubro","Novembro","Dezembro"]
consumo = [0.277,0.195,0.140,0.175,0.210,0.233,0.316,0.249,0.232,0.270,0.231,0.258] #Previsão de carga determinística para os próximos 2 anos
#PLDprev = [242.72,165.98,109.02,132.63,218.70,336.99,583.88,583.88,577.37,249.36,88.10,66.67] #Previsão determinística do PLD para os próximos 2 anos
pld =[500]*12
#previsões probabilísticas, são melhorias a serem implementadas, levando em consideração o Var.

In [ ]:

####PASSO 1
print("PASSO 1")
parametro=0.12 #margem de porcentagem, quanto o consumo pode variar em relação ao contrato inical definido, que nesse caso vai ser a média
media=np.mean(consumo) #média geral, tentar mediana ou moda
distp=[]
sazom=[]
sazo=[]
dif=[]
print(round(media,3))

index = 0
for a in consumo:
  dist=(media-consumo[index])/media
  distp.append(round(dist,2)) 
  if abs(dist) > parametro:
    dif.append(round(media-consumo[index],3))
    sazom.append(mes[index])
    sazo.append(index+1)
  index += 1
  
print((distp)) #variações de cada mês em %, em relação a media

#fazer o tratamento do montante do contrato



In [ ]:
#PASSO 2 - sobras e deficit
print("PASSO 2")
print((dif))
print((sazom))

deficit=[] #meses que faltam volume em relação a margem definida
defm=[]
sobra=[] #meses que sobram volume em relação a margem definida
sobm=[]
c=0
for b in dif:
  if b < 0:
    deficit.append(b)
    defm.append(sazom[dif.index(b)])
  else:
    sobra.append(b)
    sobm.append(sazom[dif.index(b)])
  c+=1

print((deficit))
print((defm))
print((sobra))
print(sobm)

#ordem prioridade deficit
ordemPrioridadeDeficit=sorted(deficit)
deficitOrdem=[]
for d in ordemPrioridadeDeficit:
  b=defm[deficit.index(d)]
  deficitOrdem.append(b)
  print(b)
  
#ordem prioridade sobra
ordemPrioridadeSobra=sorted(sobra,reverse=True)
sobraOrdem=[]
for d in ordemPrioridadeSobra:
  b=sobm[sobra.index(d)]
  sobraOrdem.append(b)
  print(b)



In [ ]:
#PASSO 3.0
print("PASSO 3.0")
#prioridade é o déficit pra n ficar com inadimplência de lastro

print(deficitOrdem)
print(sobraOrdem)
distrib=[]
print(len(deficitOrdem))
print(len(sobraOrdem)) # se não suprir o suficiente, aumentar o contrato base

i=0
while i < len(sobraOrdem) : #aloca cada deficit para cada sobra, o deficit é priorizado, pois é mais prejuficial ficar com insuficiencia de lastro, ao invés de liquidar na ccee
  pp=(deficitOrdem[i],sobraOrdem[i])
  distrib.append(pp)
  i+=1

print(distrib)




In [ ]:
#PASSO 4.0
print("PASSO 4.0")
#matriz sazo
#restrições e validações
sazonalidade=[0]*12

ii=0
for a in distrib:
  u=distrib[ii][0]
  o=distrib[ii][1]
  hu=mesH[mes.index(u)]
  ho=mesH[mes.index(o)]
  if hu == ho:
    sazonalidade[mes.index(u)] = 1
    sazonalidade[mes.index(o)] = -1
  elif hu<ho:
    sazonalidade[mes.index(u)] = round(hu/ho,2) #confirmar com weliton
    sazonalidade[mes.index(o)] = round(-ho/hu,2) #confirmar com weliton
  else:
    sazonalidade[mes.index(u)] = round(-hu/ho,2) #confirmar com weliton
    sazonalidade[mes.index(o)] = round(ho/hu,2) #confirmar com weliton
  ii+=1

iii=0
for s in sazonalidade:
  print(mes[iii]+":"+str(s))
  iii+=1

print(sum(sazonalidade)) #confirmar com weliton

#conferencia sazo
#m=0
#conf=0
#for s in sazo:
 #   conf = [s + ss for ss in sazo]
    #m+=1
#end for
#print(conf)

#if conf != 0:
  #print("Sazo ok")

#if conf == 0:
   # print("Sazo nok",conf)


In [ ]:
#função gerar matriz vazia

def gerar_matriz (n_linhas, n_colunas):
    matriz = []

    for _ in range(n_linhas):
        matriz.append( [0] * n_colunas )

    return matriz


In [ ]:
#PASSO 5.0
print("PASSO 5.0")

#pegar a matriz sazonalidade e obter novo montante anual
#com o novo consumo fazer o balanço para flex de 5% até 60% com incremento de 5%
#obter o custo anual e add numa matriz resultado

import xlrd
precos = xlrd.open_workbook("preco.xls")
sh = precos.sheet_by_index(0)

sazop=[0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6]
flexp=[0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6]
contrato = media
matrizcusto= gerar_matriz(len(flexp),len(sazop))
ano=[]
print(pld)

i=0
j=0
sh.cell_value(rowx=0, colx=0)
for s in range(sh.ncols): #sazo//coluna
  for f in range(sh.nrows): #flex//linha
    preco = sh.cell_value(s,f)
    index=0
    for c in consumo:
      contratos=contrato#*(1+sazonalidade[index])
      flex = f
      takemax = contratos * (1+flex) #montante maximo contratado a ser faturado
      takemin = contratos * (1-flex) #montante minimo contratado a ser faturado
      if takemin > consumo[index] < takemax: #consumo abaixo da flexibilidade inferior, possibilidade de venda
        fat = takemin * preco
        spot = (takemin - consumo[index]) * (pld[index])
        total = fat - spot
        ano.append(round(total,2))
      #end if  
      elif takemin > consumo[index] > takemax: #consumo acima da flexibilidade superior, exposição, necessário compra
        fat = takemax * preco
        spot = (takemax - consumo[index]) * (pld[index])
        total = fat + spot
        ano.append(round(total,2))
      #end if  
      else:
        #consumo dentro da flexibilidade, faturar apenas o consumo
        fat = consumo[index] * preco
        ano.append(round(fat,2))
      index += 1
    matrizcusto[f][s]=sum(ano)
    j+=1
  i+=1  
      

print(matrizcusto) #exportar csv


In [ ]:
#PASSO 6
#obter matriz resultado, 3 menores valores
#considerações